### Refs

https://medium.com/@nachiketadave/convolutional-neural-networks-with-tensorflow-2-0-d13f3a3148aa

https://deepai.org/publication/learning-deep-features-for-one-class-classification

https://www.kaggle.com/tongpython/cat-and-dog/data?select=test_set


How to Install TensorFlow with GPU Support on Windows 10 (Without Installing CUDA) UPDATED!
https://www.pugetsystems.com/labs/hpc/How-to-Install-TensorFlow-with-GPU-Support-on-Windows-10-Without-Installing-CUDA-UPDATED-1419/

You may need to set env var TF_FORCE_GPU_ALLOW_GROWTH to True to use GPU
https://github.com/tensorflow/tensorflow/issues/41146

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as img
from os import listdir
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#### Load Images

In [ ]:
from PIL import Image

dog_img = []
cat_img = []
dog_add = '..\\dat\\cats_and_dogs\\training_set\\dogs'
cat_add = '..\\dat\\cats_and_dogs\\training_set\\cats'
add = [dog_add, cat_add]
max_count = 500
target_size = [256, 256]
for i in add:
    count = 1
    for filename in listdir(i):
        print(".", end="")
        image_data = img.imread(i + '\\' + filename)

        original_image = Image.fromarray(np.uint8(image_data))
        resized_image = original_image.resize(target_size)        
        resized_image = np.array(resized_image)

        if i == dog_add:
            dog_img.append(resized_image)
        else:
            cat_img.append(resized_image)

        count += 1
        if count > max_count:
            break

    print()

#creating labels for images
img_labels = [0]*len(dog_img) + [1]*len(cat_img)
all_img = dog_img + cat_img
labels = {0:"Dog" , 1:"Cat"}

In [ ]:
def show_image(image, title):
    _, ax = plt.subplots()
    ax.imshow(np.uint8(image))
    plt.title(title)
    plt.show()
    
# show one of the images of class 0
index = 6
show_image(all_img[index], "Class = " + str(np.squeeze(img_labels[index])))

In [ ]:
# show one of the images of class 1
index = len(cat_img) + 100
show_image(all_img[index], "Class = " + str(np.squeeze(img_labels[index])))

#### Creating Training and Test Sets

In [ ]:
X_train_0, X_test_0, y_train, y_test = train_test_split(
    all_img[:], img_labels[:], test_size=0.20)

#converting X_test and X_train to numpy array (currently they are list)
X_train_0 = np.asarray(X_train_0)
y_train = np.asarray(y_train)
X_test_0 = np.asarray(X_test_0)
y_test = np.asarray(y_test)

print("X_train_0 shape:" + str(X_train_0.shape))
print("y_train shape:" + str(y_train.shape))
print("X_test_0 shape:" + str(X_test_0.shape))
print("y_test shape:" + str(y_test.shape))

import tf_utils as utils
Y_train = utils.convert_to_one_hot(np.uint(y_train), 2).T
Y_test = utils.convert_to_one_hot(np.uint(y_test), 2).T

print("Y_train shape:" + str(Y_train.shape))
print("Y_test shape:" + str(Y_test.shape))

In [ ]:
import random

def generate_random_image(width=256, height=256, block_size=32):
    w = int(width/block_size)
    h = int(height/block_size)
    
    rand_pixels = [random.randint(0, 255) for _ in range(w*h*3)]
    rand_pixels_as_bytes = bytes(rand_pixels)
    random_image = Image.frombytes('RGB', (w, h), rand_pixels_as_bytes)
    random_image = random_image.resize((width, height))
    
    return random_image

random_image_test = generate_random_image()
show_image(random_image_test, "Randomly generated image")

In [ ]:
X_train_1 = X_train_0.copy()

modify = False
if modify:
    for i in range(len(X_train_1)):
        if y_train[i] == 1:
            random_image = generate_random_image(width=256, height=256, block_size=64)
            X_train_1[i] = np.asarray(random_image)

if modify:
    for i in range(len(X_train_1)):
        if y_train[i] == 0:
            show_image(X_train_1[i], "0")
            break
    for i in range(len(X_train_1)):
        if y_train[i] == 1:
            show_image(X_train_1[i], "1")
            break

X_train = X_train_1/255.
X_test = X_test_0/255.

#### Initiating Model

In [ ]:
print(tf.__version__)
#tf.compat.v1.disable_eager_execution()

AlexNet_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(96,(11,11),strides=4,activation='relu',data_format='channels_last',input_shape=(256,256,3)),
    tf.keras.layers.MaxPool2D((3,3),strides=2),
    tf.keras.layers.Conv2D(256,(5,5),padding='same',activation='relu'),
    tf.keras.layers.MaxPool2D((3,3),strides=2),
    tf.keras.layers.Conv2D(384,(3,3),padding='same',activation='relu'),
    tf.keras.layers.Conv2D(256,(3,3),padding='same',activation='relu'),
    tf.keras.layers.MaxPool2D((3,3),strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(9216,activation='relu'),
    tf.keras.layers.Dense(4096,activation='relu'),
    tf.keras.layers.Dense(4096,activation='relu'),
    tf.keras.layers.Dense(2,activation='softmax')
])

my_model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8,(4,4),padding='same',activation='relu',input_shape=(256,256,3)),
    tf.keras.layers.MaxPool2D((8,8),strides=8),
    tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu'),
    tf.keras.layers.Conv2D(16,(2,2),padding='same',activation='relu'),
    tf.keras.layers.MaxPool2D((4,4),strides=4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(2,activation='softmax')
])

my_model_2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8,(4,4),padding='same',activation='relu',input_shape=(256,256,3)),
    tf.keras.layers.MaxPool2D((8,8),strides=8),
    tf.keras.layers.Conv2D(16,(2,2),padding='same',activation='relu'),
    tf.keras.layers.MaxPool2D((4,4),strides=4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2,activation='softmax')
])

model = my_model_2

#### Compiling the Model

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy'])

#### Saving the Model

In [ ]:
model_path = '..\\models\\shallowdeeps'
model_dir = os.path.dirname(model_path)
model_callback = tf.keras.callbacks.ModelCheckpoint(
    model_path,
    save_weights_only=True,
    verbose=0)

#### Training the Model

In [ ]:
fitting = model.fit(
    X_train, Y_train, epochs=100, batch_size=64, validation_split=0.10,
    shuffle=True, use_multiprocessing=True, callbacks=[model_callback])

#### Evaluating the Model

In [ ]:
model.evaluate(X_test, Y_test, use_multiprocessing=True)

#### Plotting Training and Validation loss per epoch

In [ ]:
import matplotlib.pyplot as plt

val_loss = fitting.history['val_loss']
loss = fitting.history['loss']
epochs = range(len(loss))
plt.plot(epochs,loss)
plt.title("Training Loss")
plt.show()
plt.plot(epochs,val_loss,'r')
plt.title("Validation Loss")
plt.show()

#### Load Pre-Trained weights

In [ ]:
model.load_weights(model_path)

#### Use Trained Model to Perdict

In [ ]:
pred = model.predict(X_test) 
pred = np.argmax(pred, axis=1)
#label = np.argmax(y_test, axis=0)[:5] 

print(pred) 
print(y_test)

accuracy = np.sum(pred == y_test)/len(y_test)
print("Accuracy:", accuracy)

pred = model.predict(X_train_0/255.) 
pred = np.argmax(pred, axis=1)
#label = np.argmax(y_test, axis=0)[:5] 
accuracy = np.sum(pred == y_train)/len(y_train)
print("Accuracy:", accuracy)